In [1]:
!conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/processed/alura_real_estate.csv')
df.head()

,square_meters,year_built,garage_cars,price
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [5]:
x = df.drop('price', axis=1)
y = df['price']

In [6]:
!conda install scikit-learn -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [8]:
X_train.head()

,square_meters,year_built,garage_cars
135,156.0,1970,2
1452,100.0,2005,2
762,144.0,2009,2
932,177.0,2006,3
435,154.0,1996,2


In [9]:
y_train

135     174000
1452    145000
762     215200
932     320000
435     212000
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: price, Length: 1022, dtype: int64

# Linear Regression

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
lr.predict(X_test)[0]

119279.77015440399

In [14]:
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
lr_predicted = lr.predict(X_test)

In [16]:
import math

mse = mean_squared_error(y_test, lr_predicted)
r2 = r2_score(y_test, lr_predicted)
rmse = math.sqrt(mse)

In [17]:
print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')

 rmse: 45592.399782518485, mse: 2078666917.9289918, r2 0.7021153642898047


# XGBoost

In [18]:
!conda install xgboost -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/julio/anaconda3/envs/py10

  added / updated specs:
    - xgboost


The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  pkgs/main/linux-64::_py-xgboost-mutex-2.0-cpu_0 None
  libxgboost         pkgs/main/linux-64::libxgboost-1.7.3-h6a678d5_0 None
  py-xgboost         pkgs/main/linux-64::py-xgboost-1.7.3-py310h06a4308_0 None
  xgboost            pkgs/main/linux-64::xgboost-1.7.3-py310h06a4308_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


In [19]:
from xgboost import XGBRegressor

In [20]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [22]:
xgb_predicted = xgb.predict(X_test)

In [23]:
mse = mean_squared_error(y_test, xgb_predicted)
r2 = r2_score(y_test, xgb_predicted)
rmse = math.sqrt(mse)
print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')

 rmse: 39380.40189141977, mse: 1550816053.129738, r2 0.7777593557411607


## Testing params

In [24]:
xgb = XGBRegressor(learning_rate=0.2, n_estimators=50, random_stage=42)
xgb.fit(X_train, y_train)

[19:10:21] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:767: 
Parameters: { "random_stage" } are not used.



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=50, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_stage=42, ...)

In [25]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test, xgb_predicted)
r2 = r2_score(y_test, xgb_predicted)
rmse = math.sqrt(mse)
print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')

 rmse: 37238.789724353286, mse: 1386727460.1346002, r2 0.8012741720529797
